## Supervised Learning on Simulations on all Retro Aldol Simulations

These will be run one after the other

In [1]:
import sys # note temporary... 
sys.path.append("..") # note temporary...

import re 
import pandas as pd
from key_interactions_finder import pycontact_processing
from key_interactions_finder import data_preperation
from key_interactions_finder import model_building
from key_interactions_finder import post_proccessing
from key_interactions_finder import pymol_projections

## 4a2s_RA95_5

In [4]:
# all input files 
in_dir = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\RA_Workup\Raw_Datasets\4a2s_RA95_5"
out_dir = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\RA_Workup\Raw_Results\4a2s_RA95_5"

pycontact_files_horizontal = ["PyContact_Per_Frame_Interactions_Block1.csv", "PyContact_Per_Frame_Interactions_Block2.csv",
                              "PyContact_Per_Frame_Interactions_Block3.csv", "PyContact_Per_Frame_Interactions_Block4.csv",
                              "PyContact_Per_Frame_Interactions_Block5.csv", "PyContact_Per_Frame_Interactions_Block6.csv",
                              "PyContact_Per_Frame_Interactions_Block7.csv", "PyContact_Per_Frame_Interactions_Block8.csv",
                              "PyContact_Per_Frame_Interactions_Block9.csv", "PyContact_Per_Frame_Interactions_Block10.csv",
                              "PyContact_Per_Frame_Interactions_Block11.csv", "PyContact_Per_Frame_Interactions_Block12.csv",
                              "PyContact_Per_Frame_Interactions_Block13.csv", "PyContact_Per_Frame_Interactions_Block14.csv",
                              "PyContact_Per_Frame_Interactions_Block15.csv", "PyContact_Per_Frame_Interactions_Block16.csv",
                              "PyContact_Per_Frame_Interactions_Block17.csv"]

cat_distances = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\RA_Workup\Raw_Datasets\4a2s_RA95_5\Cat_Distance_4a2s_RA95_5.dat"


# dataset prep parts. 
pycontact_dataset = pycontact_processing.PyContactInitializer(
    pycontact_files=pycontact_files_horizontal,
    multiple_files=True,
    merge_files_method="horizontal",  
    remove_false_interactions=True,
    in_dir=in_dir,
)

supervised_dataset = data_preperation.SupervisedFeatureData(
    input_df=pycontact_dataset.prepared_df,
    target_file=cat_distances,
    is_classification=False,
    header_present=True 
)

# Filtering 
supervised_dataset.reset_filtering()
supervised_dataset.filter_by_occupancy(min_occupancy=25)
supervised_dataset.filter_by_avg_strength(average_strength_cut_off=1.0)


# Instantiate the model.
ml_model = model_building.RegressionModel(
    dataset=supervised_dataset.df_filtered,
    evaluation_split_ratio=0.15,
    models_to_use=["CatBoost", "XGBoost"], # "Random_Forest"
    scaling_method="min_max",
    out_dir=out_dir, 
    cross_validation_splits=5, 
    cross_validation_repeats=3,
    search_approach="none",
)

Your PyContact file(s) have been succefully processed.
You have 3056 features and 10000 observations.
The fully processed dataframe is accesible from the '.prepared_df' class attribute.
Your PyContact features and target variable have been succesufully merged.
You can access this dataset through the class attribute: '.df_processed'.

Below is a summary of the machine learning you have planned.
You will use 5-fold cross validation and perform 3 repeats.
You will use up to 916 features to build each model, with 85.0% of your data used for training the model, which is 8500 observations. 
15.0% of your data will be used for evaluating the best models produced by the 5-fold cross validation, which is 1500 observations.
You have selected to build 2 machine learning model(s), with the following hyperparameters: 
 
A CatBoost model, with grid search parameters: 
{'iterations': [100]} 

A XGBoost model, with grid search parameters: 
{'n_estimators': [100]} 

If you're happy with the above, lets

In [5]:
ml_model.build_models(save_models=True)

Learning rate set to 0.360343
0:	learn: 1.3004190	total: 110ms	remaining: 10.9s
1:	learn: 1.1389072	total: 196ms	remaining: 9.61s
2:	learn: 1.0301103	total: 282ms	remaining: 9.1s
3:	learn: 0.9672082	total: 361ms	remaining: 8.65s
4:	learn: 0.9209676	total: 463ms	remaining: 8.79s
5:	learn: 0.8815239	total: 546ms	remaining: 8.55s
6:	learn: 0.8508971	total: 625ms	remaining: 8.31s
7:	learn: 0.8128166	total: 725ms	remaining: 8.34s
8:	learn: 0.7868387	total: 808ms	remaining: 8.17s
9:	learn: 0.7682436	total: 891ms	remaining: 8.02s
10:	learn: 0.7534889	total: 993ms	remaining: 8.03s
11:	learn: 0.7481171	total: 1.09s	remaining: 7.99s
12:	learn: 0.7363876	total: 1.17s	remaining: 7.86s
13:	learn: 0.7300522	total: 1.25s	remaining: 7.7s
14:	learn: 0.7180372	total: 1.36s	remaining: 7.71s
15:	learn: 0.7136690	total: 1.46s	remaining: 7.66s
16:	learn: 0.7091090	total: 1.54s	remaining: 7.52s
17:	learn: 0.7036514	total: 1.62s	remaining: 7.38s
18:	learn: 0.6975783	total: 1.71s	remaining: 7.27s
19:	learn: 0.

,model,best_params,best_score,best_standard_deviation
0,CatBoost,{'iterations': 100},0.804701,0.010508
1,XGBoost,{'n_estimators': 100},0.798651,0.011695


In [10]:
reports = ml_model.evaluate_models()
reports

,Model,Explained Variance,Mean Absolute Error,MSE,RMSE,Mean Squared Log Error,r squared
0,CatBoost,0.8265,0.4978,0.4423,0.6651,0.0166,0.8265
1,XGBoost,0.8091,0.5170,0.4872,0.6980,0.0179,0.8089


In [11]:
post_proc = post_proccessing.SupervisedPostProcessor(
    out_dir=out_dir,
)
post_proc.load_models_from_instance(supervised_model=ml_model)

post_proc.get_feature_importance()
post_proc.get_per_res_importance()

# Pymol Results
pymol_projections.project_multiple_per_res_scores(
    all_per_res_scores=post_proc.all_per_residue_scores,
    out_dir=out_dir
)

pymol_projections.project_multiple_per_feature_scores(
    all_feature_scores=post_proc.all_feature_importances,
    numb_features="all",
    out_dir=out_dir
)

C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\RA_Workup\Raw_Results\4a2s_RA95_5/CatBoost_Feature_Importances.csv written to disk.
C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\RA_Workup\Raw_Results\4a2s_RA95_5/XGBoost_Feature_Importances.csv written to disk.
All feature importances written to disk.
C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\RA_Workup\Raw_Results\4a2s_RA95_5/CatBoost_Per_Residue_Importances.csv written to disk.
C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\RA_Workup\Raw_Results\4a2s_RA95_5/XGBoost_Per_Residue_Importances.csv written to disk.
All per residue feature importance scores were saved to disk.
The file: C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\RA_Workup\Raw_Results\4a2s_RA95_5/C

## 4a2r_RA95_5_5

In [12]:
# all input files 
in_dir = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\RA_Workup\Raw_Datasets\4a2r_RA95_5_5"
out_dir = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\RA_Workup\Raw_Results\4a2r_RA95_5_5"

pycontact_files_horizontal = ["PyContact_Per_Frame_Interactions_Block1.csv", "PyContact_Per_Frame_Interactions_Block2.csv",
                              "PyContact_Per_Frame_Interactions_Block3.csv", "PyContact_Per_Frame_Interactions_Block4.csv",
                              "PyContact_Per_Frame_Interactions_Block5.csv", "PyContact_Per_Frame_Interactions_Block6.csv",
                              "PyContact_Per_Frame_Interactions_Block7.csv", "PyContact_Per_Frame_Interactions_Block8.csv",
                              "PyContact_Per_Frame_Interactions_Block9.csv", "PyContact_Per_Frame_Interactions_Block10.csv",
                              "PyContact_Per_Frame_Interactions_Block11.csv", "PyContact_Per_Frame_Interactions_Block12.csv",
                              "PyContact_Per_Frame_Interactions_Block13.csv", "PyContact_Per_Frame_Interactions_Block14.csv",
                              "PyContact_Per_Frame_Interactions_Block15.csv", "PyContact_Per_Frame_Interactions_Block16.csv",
                              "PyContact_Per_Frame_Interactions_Block17.csv"]

cat_distances = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\RA_Workup\Raw_Datasets\4a2r_RA95_5_5\Cat_Distance_4a2r_RA95_5_5.dat"


# dataset prep parts. 
pycontact_dataset = pycontact_processing.PyContactInitializer(
    pycontact_files=pycontact_files_horizontal,
    multiple_files=True,
    merge_files_method="horizontal",  
    remove_false_interactions=True,
    in_dir=in_dir,
)

supervised_dataset = data_preperation.SupervisedFeatureData(
    input_df=pycontact_dataset.prepared_df,
    target_file=cat_distances,
    is_classification=False,
    header_present=True 
)

# Filtering 
supervised_dataset.reset_filtering()
supervised_dataset.filter_by_occupancy(min_occupancy=25)
supervised_dataset.filter_by_avg_strength(average_strength_cut_off=1.0)


# Instantiate the model.
ml_model = model_building.RegressionModel(
    dataset=supervised_dataset.df_filtered,
    evaluation_split_ratio=0.15,
    models_to_use=["CatBoost", "XGBoost"], # "Random_Forest"
    scaling_method="min_max",
    out_dir=out_dir, 
    cross_validation_splits=5, 
    cross_validation_repeats=3,
    search_approach="none",
)

Your PyContact file(s) have been succefully processed.
You have 2731 features and 10000 observations.
The fully processed dataframe is accesible from the '.prepared_df' class attribute.
Your PyContact features and target variable have been succesufully merged.
You can access this dataset through the class attribute: '.df_processed'.

Below is a summary of the machine learning you have planned.
You will use 5-fold cross validation and perform 3 repeats.
You will use up to 916 features to build each model, with 85.0% of your data used for training the model, which is 8500 observations. 
15.0% of your data will be used for evaluating the best models produced by the 5-fold cross validation, which is 1500 observations.
You have selected to build 2 machine learning model(s), with the following hyperparameters: 
 
A CatBoost model, with grid search parameters: 
{'iterations': [100]} 

A XGBoost model, with grid search parameters: 
{'n_estimators': [100]} 

If you're happy with the above, lets

In [13]:
ml_model.build_models(save_models=True)

Learning rate set to 0.360343
0:	learn: 1.1133893	total: 125ms	remaining: 12.4s
1:	learn: 0.9198986	total: 208ms	remaining: 10.2s
2:	learn: 0.8303210	total: 293ms	remaining: 9.48s
3:	learn: 0.7638076	total: 376ms	remaining: 9.03s
4:	learn: 0.7245323	total: 472ms	remaining: 8.96s
5:	learn: 0.6908080	total: 570ms	remaining: 8.93s
6:	learn: 0.6674370	total: 671ms	remaining: 8.91s
7:	learn: 0.6547987	total: 795ms	remaining: 9.14s
8:	learn: 0.6440841	total: 895ms	remaining: 9.05s
9:	learn: 0.6378545	total: 990ms	remaining: 8.91s
10:	learn: 0.6293799	total: 1.08s	remaining: 8.76s
11:	learn: 0.6238026	total: 1.18s	remaining: 8.62s
12:	learn: 0.6171866	total: 1.26s	remaining: 8.44s
13:	learn: 0.6114781	total: 1.34s	remaining: 8.24s
14:	learn: 0.6060046	total: 1.45s	remaining: 8.22s
15:	learn: 0.5993319	total: 1.54s	remaining: 8.11s
16:	learn: 0.5936345	total: 1.63s	remaining: 7.94s
17:	learn: 0.5880664	total: 1.74s	remaining: 7.9s
18:	learn: 0.5824132	total: 1.82s	remaining: 7.75s
19:	learn: 0

,model,best_params,best_score,best_standard_deviation
0,CatBoost,{'iterations': 100},0.798014,0.010131
1,XGBoost,{'n_estimators': 100},0.795223,0.010382


In [14]:
reports = ml_model.evaluate_models()
reports

,Model,Explained Variance,Mean Absolute Error,MSE,RMSE,Mean Squared Log Error,r squared
0,CatBoost,0.7996,0.4698,0.3711,0.6092,0.0129,0.7995
1,XGBoost,0.7881,0.4641,0.3925,0.6265,0.0133,0.7879


In [15]:
post_proc = post_proccessing.SupervisedPostProcessor(
    out_dir=out_dir,
)
post_proc.load_models_from_instance(supervised_model=ml_model)

post_proc.get_feature_importance()
post_proc.get_per_res_importance()

# Pymol Results
pymol_projections.project_multiple_per_res_scores(
    all_per_res_scores=post_proc.all_per_residue_scores,
    out_dir=out_dir
)

pymol_projections.project_multiple_per_feature_scores(
    all_feature_scores=post_proc.all_feature_importances,
    numb_features="all",
    out_dir=out_dir
)

C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\RA_Workup\Raw_Results\4a2r_RA95_5_5/CatBoost_Feature_Importances.csv written to disk.
C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\RA_Workup\Raw_Results\4a2r_RA95_5_5/XGBoost_Feature_Importances.csv written to disk.
All feature importances written to disk.
C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\RA_Workup\Raw_Results\4a2r_RA95_5_5/CatBoost_Per_Residue_Importances.csv written to disk.
C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\RA_Workup\Raw_Results\4a2r_RA95_5_5/XGBoost_Per_Residue_Importances.csv written to disk.
All per residue feature importance scores were saved to disk.
The file: C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\RA_Workup\Raw_Results\4a2r_

## 5an7_RA95_5_8F

In [16]:
# all input files 
in_dir = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\RA_Workup\Raw_Datasets\5an7_RA95_5_8F"
out_dir = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\RA_Workup\Raw_Results\5an7_RA95_5_8F"

pycontact_files_horizontal = ["PyContact_Per_Frame_Interactions_Block1.csv", "PyContact_Per_Frame_Interactions_Block2.csv",
                              "PyContact_Per_Frame_Interactions_Block3.csv", "PyContact_Per_Frame_Interactions_Block4.csv",
                              "PyContact_Per_Frame_Interactions_Block5.csv", "PyContact_Per_Frame_Interactions_Block6.csv",
                              "PyContact_Per_Frame_Interactions_Block7.csv", "PyContact_Per_Frame_Interactions_Block8.csv",
                              "PyContact_Per_Frame_Interactions_Block9.csv", "PyContact_Per_Frame_Interactions_Block10.csv",
                              "PyContact_Per_Frame_Interactions_Block11.csv", "PyContact_Per_Frame_Interactions_Block12.csv",
                              "PyContact_Per_Frame_Interactions_Block13.csv", "PyContact_Per_Frame_Interactions_Block14.csv",
                              "PyContact_Per_Frame_Interactions_Block15.csv", "PyContact_Per_Frame_Interactions_Block16.csv",
                              "PyContact_Per_Frame_Interactions_Block17.csv"]

cat_distances = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\RA_Workup\Raw_Datasets\5an7_RA95_5_8F\Cat_Distance_5an7_RA95_5_8F.dat"


# dataset prep parts. 
pycontact_dataset = pycontact_processing.PyContactInitializer(
    pycontact_files=pycontact_files_horizontal,
    multiple_files=True,
    merge_files_method="horizontal",  
    remove_false_interactions=True,
    in_dir=in_dir,
)

supervised_dataset = data_preperation.SupervisedFeatureData(
    input_df=pycontact_dataset.prepared_df,
    target_file=cat_distances,
    is_classification=False,
    header_present=True 
)

# Filtering 
supervised_dataset.reset_filtering()
supervised_dataset.filter_by_occupancy(min_occupancy=25)
supervised_dataset.filter_by_avg_strength(average_strength_cut_off=1.0)


# Instantiate the model.
ml_model = model_building.RegressionModel(
    dataset=supervised_dataset.df_filtered,
    evaluation_split_ratio=0.15,
    models_to_use=["CatBoost", "XGBoost"], # "Random_Forest"
    scaling_method="min_max",
    out_dir=out_dir, 
    cross_validation_splits=5, 
    cross_validation_repeats=3,
    search_approach="none",
)

Your PyContact file(s) have been succefully processed.
You have 2923 features and 10000 observations.
The fully processed dataframe is accesible from the '.prepared_df' class attribute.
Your PyContact features and target variable have been succesufully merged.
You can access this dataset through the class attribute: '.df_processed'.

Below is a summary of the machine learning you have planned.
You will use 5-fold cross validation and perform 3 repeats.
You will use up to 907 features to build each model, with 85.0% of your data used for training the model, which is 8500 observations. 
15.0% of your data will be used for evaluating the best models produced by the 5-fold cross validation, which is 1500 observations.
You have selected to build 2 machine learning model(s), with the following hyperparameters: 
 
A CatBoost model, with grid search parameters: 
{'iterations': [100]} 

A XGBoost model, with grid search parameters: 
{'n_estimators': [100]} 

If you're happy with the above, lets

In [17]:
ml_model.build_models(save_models=True)

Learning rate set to 0.360343
0:	learn: 1.2148315	total: 122ms	remaining: 12.1s
1:	learn: 1.1116832	total: 202ms	remaining: 9.9s
2:	learn: 1.0499096	total: 304ms	remaining: 9.82s
3:	learn: 0.9874814	total: 390ms	remaining: 9.36s
4:	learn: 0.9501630	total: 475ms	remaining: 9.02s
5:	learn: 0.9286456	total: 556ms	remaining: 8.71s
6:	learn: 0.9030824	total: 676ms	remaining: 8.98s
7:	learn: 0.8783306	total: 759ms	remaining: 8.73s
8:	learn: 0.8615636	total: 854ms	remaining: 8.63s
9:	learn: 0.8467627	total: 937ms	remaining: 8.43s
10:	learn: 0.8299360	total: 1.01s	remaining: 8.21s
11:	learn: 0.8160193	total: 1.09s	remaining: 8.02s
12:	learn: 0.8033716	total: 1.18s	remaining: 7.87s
13:	learn: 0.7928293	total: 1.25s	remaining: 7.7s
14:	learn: 0.7841316	total: 1.33s	remaining: 7.55s
15:	learn: 0.7788000	total: 1.41s	remaining: 7.42s
16:	learn: 0.7723673	total: 1.5s	remaining: 7.3s
17:	learn: 0.7653795	total: 1.58s	remaining: 7.19s
18:	learn: 0.7581064	total: 1.67s	remaining: 7.13s
19:	learn: 0.75

,model,best_params,best_score,best_standard_deviation
0,CatBoost,{'iterations': 100},0.692833,0.015051
1,XGBoost,{'n_estimators': 100},0.681055,0.017946


In [18]:
reports = ml_model.evaluate_models()
reports

,Model,Explained Variance,Mean Absolute Error,MSE,RMSE,Mean Squared Log Error,r squared
0,CatBoost,0.7395,0.5220,0.5222,0.7226,0.0206,0.7391
1,XGBoost,0.7205,0.5257,0.5595,0.7480,0.0225,0.7205


In [19]:
post_proc = post_proccessing.SupervisedPostProcessor(
    out_dir=out_dir,
)
post_proc.load_models_from_instance(supervised_model=ml_model)

post_proc.get_feature_importance()
post_proc.get_per_res_importance()

# Pymol Results
pymol_projections.project_multiple_per_res_scores(
    all_per_res_scores=post_proc.all_per_residue_scores,
    out_dir=out_dir
)

pymol_projections.project_multiple_per_feature_scores(
    all_feature_scores=post_proc.all_feature_importances,
    numb_features="all",
    out_dir=out_dir
)

C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\RA_Workup\Raw_Results\5an7_RA95_5_8F/CatBoost_Feature_Importances.csv written to disk.
C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\RA_Workup\Raw_Results\5an7_RA95_5_8F/XGBoost_Feature_Importances.csv written to disk.
All feature importances written to disk.
C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\RA_Workup\Raw_Results\5an7_RA95_5_8F/CatBoost_Per_Residue_Importances.csv written to disk.
C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\RA_Workup\Raw_Results\5an7_RA95_5_8F/XGBoost_Per_Residue_Importances.csv written to disk.
All per residue feature importance scores were saved to disk.
The file: C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\RA_Workup\Raw_Results\5